# Biigle parser for annotators

This notebook is used to parse the annotations from the Biigle reports.

It allows you to export the reports, find MaxN, calculate sizes...

The notebook walks you through the process. Fill in the values in the next two cells, and run the remaining cells.


If there are any issues, or if it breaks please write an email to Kalindi, or open an issue on: 
https://github.com/wildlifeai/Spyfish-Aotearoa-toolkit/issues


In [ ]:
# The ID of the video volume you want to export annotations from.
# You can find it on the url on biigle when you are on the page with all the clips:
# For example: https://biigle.de/volumes/25173, 25173 is the volume id.
# VOLUME_ID = "25173" # no sizes (yet.)
VOLUME_ID = "25516" # has sizes

In [ ]:

# If you have the biigle credentials set up as env variables, you can leave them as None.

# Your Biigle account
BIIGLE_API_EMAIL = None
# BIIGLE_API_EMAIL="kiwia@wildlife.ai"

# A token is a special password like number used for the Biigle API. 
# Find yours here: https://biigle.de/settings/tokens
# Keep this secret.
BIIGLE_API_TOKEN = None
# BIIGLE_API_TOKEN = "Mag1CN0"



In [ ]:
# Loading and installing the necessary libraries
try:
    from sftk.utils import ping
except ImportError:
    print("Downloading the Spyfish Aotearoa toolkit...")
    !pip install --upgrade --no-deps -q git+https://github.com/wildlifeai/Spyfish-Aotearoa-toolkit.git



from sftk.biigle_parser import BiigleParser

In [ ]:

biigle_parser = BiigleParser(email=BIIGLE_API_EMAIL, token=BIIGLE_API_TOKEN)

In [ ]:
processed_annotations = biigle_parser.process_video_annotations(VOLUME_ID)
# Max counts every 30 seconds
max_n_30s_df = processed_annotations["max_n_30s_df"]
# Max count of whole video (used to determine where in the videos to annotate for size)
max_n_df = processed_annotations["max_n_df"]
# Potentially empty, if it was the video was not annotated for size.
sizes_df = processed_annotations["sizes_df"]

## Review the parsed dataframes

In [ ]:
max_n_30s_df.head()

In [ ]:
max_n_df.head()

In [ ]:

sizes_df.head()

In [ ]:
# Export the processed annotations into CVS files

max_n_30s_df.to_csv(f"{VOLUME_ID}_max_n_30s_df.csv", index=False)
max_n_df.to_csv(f"{VOLUME_ID}_max_n_df.csv", index=False)
sizes_df.to_csv(f"{VOLUME_ID}_sizes_df.csv", index=False)

In [ ]:
# Export for general annotations file
# Not needed here, just added as a fyi
final_annotations_output_df = biigle_parser.format_count_annotations_output(max_n_df)
final_annotations_output_df.columns
